<a href="https://colab.research.google.com/github/sula316/ML_MY_Project_FCUT/blob/main/ML_MY_Project_FCUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stept_1_Data_Preprocessing

##工具

In [61]:
!pip install numpy pandas matplotlib seaborn scikit-learn scipy > log.txt

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy

##Healthy

###1.讀檔>>去除取樣資料的檔頭>>取22~30023

根據檔案內容的初步檢查，數據在第二個 `***End_of_Header***` 標記之後開始。我們將解析檔案，找到實際數據開始的行，然後使用 `pandas` 讀取並篩選指定的行範圍（第22到第30023行）。

In [63]:
import pandas as pd

file_path = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_01.lvm'

def get_data_header_line_index(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    header_end_count = 0
    data_header_line_index = -1 # 0-indexed line number of the actual data header
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_end_count += 1
        if header_end_count == 2: # 在第二個 '***End_of_Header***' 之後
            # 尋找包含資料欄位名稱的行
            if 'X_Value\tX\tY\tZ\tVoltage\tComment' in line:
                data_header_line_index = i
                break
            # 處理檔頭可能在 End_of_Header 後下一行的情況
            elif i + 1 < len(lines) and lines[i+1].startswith('X_Value\tX\tY\tZ\tVoltage\tComment'):
                 data_header_line_index = i + 1
                 break
    if data_header_line_index == -1:
        raise ValueError("在檔案中找不到資料欄位標題行。")
    return data_header_line_index

data_header_0_indexed = get_data_header_line_index(file_path)

print(f"數據列標題行 (0-indexed) 位於: {data_header_0_indexed}")

# 讀取從數據列標題行 (X_Value行) 的下一行開始，直到檔案的最後一行
# 這表示我們將跳過所有標頭行，以及包含列標題的行本身。
skiprows_for_pd_read_csv = data_header_0_indexed + 1

# 使用 pandas 讀取檔案，直接從計算出的行數開始讀取到檔案結束
# 因為我們手動計算了 skiprows，所以 header=None，然後手動設置列名
df_temp = pd.read_csv(
    file_path,
    skiprows=skiprows_for_pd_read_csv,
    sep='\t',
    header=None # 不將任何行解釋為標題行，因為我們已將其跳過
)

# 從文件中讀取實際的列標題，並手動指定給 DataFrame
with open(file_path, 'r', encoding='utf-8') as f:
    header_line_content = f.readlines()[data_header_0_indexed]
# 分割並清理列名中的空格
columns = [col.strip() for col in header_line_content.split('\t') if col.strip()]

# 移除 'Comment' 列，因為它看起來是空的或不相關的
if 'Comment' in columns:
    columns.remove('Comment')

# 將處理過的列名分配給 DataFrame
df_temp.columns = columns

processed_df = df_temp.copy()

print("處理後的資料前5行：")
display(processed_df.head())
display(processed_df.tail())
print("處理後的資料形狀：")
print(processed_df.shape)

數據列標題行 (0-indexed) 位於: 22
處理後的資料前5行：


,X_Value,X,Y,Z,Voltage
0,0.000000,0.053857,-0.002894,0.013345,0.000443
1,0.000156,0.032555,0.021578,0.003450,0.002672
2,0.000312,0.004798,-0.005554,0.051723,0.002672
3,0.000469,-0.019530,-0.003355,0.032483,0.002991
4,0.000625,-0.004428,0.011777,0.012203,0.002991


,X_Value,X,Y,Z,Voltage
29995,4.686719,0.009914,0.010303,-0.003439,0.001717
29996,4.686875,0.006148,-0.000258,-0.008753,0.002035
29997,4.687031,0.003009,0.003586,0.000388,0.001398
29998,4.687187,-0.010613,-0.001900,0.007021,-0.000513
29999,4.687344,-0.000942,-0.001543,-0.002888,0.001080


處理後的資料形狀：
(30000, 5)


###2.合併資料>>Healthy的各維度>>各別組成一張Table

In [64]:
import os
import pandas as pd

# 重複使用先前定義的函數來尋找資料標題行索引
def get_data_header_line_index(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    header_end_count = 0
    data_header_line_index = -1 # 資料實際標題行的0索引行號
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_end_count += 1
        if header_end_count == 2: # 在第二個 '***End_of_Header***' 之後
            if 'X_Value\tX\tY\tZ\tVoltage\tComment' in line:
                data_header_line_index = i
                break
            elif i + 1 < len(lines) and lines[i+1].startswith('X_Value\tX\tY\tZ\tVoltage\tComment'):
                 data_header_line_index = i + 1
                 break
    if data_header_line_index == -1:
        raise ValueError(f"在檔案中找不到資料標題行: {file_path}")
    return data_header_line_index

# 定義包含 LVM 檔案的目錄 (與前一個檔案路徑的目錄相同)
data_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/'
lvm_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.lvm')]

all_processed_dfs = []

print(f"在目錄中找到 {len(lvm_files)} 個 LVM 檔案: {data_dir}\n")

for file_path in lvm_files:
    print(f"正在處理檔案: {file_path}")
    try:
        # 取得資料標題行 (X_Value, X, Y, Z, Voltage) 所在的0索引行號
        data_header_0_indexed = get_data_header_line_index(file_path)

        # 計算 pandas 需要跳過的行數: 跳過所有標頭行 + 包含資料欄位名稱的行
        skiprows_for_pd_read_csv = data_header_0_indexed + 1

        # 從標頭後的資料第一行開始讀取到檔案結束
        df_temp = pd.read_csv(
            file_path,
            skiprows=skiprows_for_pd_read_csv,
            sep='\t',
            header=None # 我們將手動指定欄位名稱
        )

        # 從資料標題行中提取實際的欄位名稱
        with open(file_path, 'r', encoding='utf-8') as f:
            header_line_content = f.readlines()[data_header_0_indexed]
        columns = [col.strip() for col in header_line_content.split('\t') if col.strip()]

        # 如果存在 'Comment' 欄位則移除，因為之前被認為不相關
        if 'Comment' in columns:
            columns.remove('Comment')

        # 將欄位名稱分配給 DataFrame
        if len(df_temp.columns) == len(columns):
            df_temp.columns = columns
        else:
            print(f"警告: {file_path} 中的欄位計數不匹配。預期 {len(columns)} 個欄位，但 DataFrame 有 {len(df_temp.columns)} 個。跳過此檔案。\n")
            continue # 如果欄位計數不匹配，則跳過到下一個檔案

        # 提取不含副檔名的檔案名稱作為欄位的前綴 (除了 'X_Value')
        filename_without_ext = os.path.basename(file_path).split('.')[0]
        renamed_cols = {col: f"{filename_without_ext}_{col}" for col in columns if col != 'X_Value'}
        df_temp = df_temp.rename(columns=renamed_cols)

        all_processed_dfs.append(df_temp)

    except Exception as e:
        print(f"處理 {file_path} 時發生錯誤: {e}\n")

merged_df = pd.DataFrame()
if all_processed_dfs:
    # 使用第一個處理過的 DataFrame 初始化 merged_df。
    # 'X_Value' 將作為合併的共同欄位。
    merged_df = all_processed_dfs[0]

    # 使用外連接 (outer join) 將後續的 DataFrames 合併到 'X_Value' 上，以保留所有時間點
    for i in range(1, len(all_processed_dfs)):
        merged_df = pd.merge(merged_df, all_processed_dfs[i], on='X_Value', how='outer')

    # 依 'X_Value' 排序以保持時間序列順序
    merged_df = merged_df.sort_values(by='X_Value').reset_index(drop=True)

    print("\n所有檔案處理和合併成功！")
    print("合併後的資料 (前5行):")
    display(merged_df.head())
    print("\n合併後的資料形狀:")
    print(merged_df.shape)
else:
    print("沒有成功處理或找到要合併的 DataFrames。")

在目錄中找到 48 個 LVM 檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/

正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_08.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_02.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_31.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_34.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_32.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_13.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_09.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_04.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_42.lvm
正在處理檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy/202409302014_2

,X_Value,202409302014_08_X,202409302014_08_Y,202409302014_08_Z,202409302014_08_Voltage,202409302014_02_X,202409302014_02_Y,202409302014_02_Z,202409302014_02_Voltage,202409302014_31_X,...,202409302014_17_Z,202409302014_17_Voltage,202409302014_11_X,202409302014_11_Y,202409302014_11_Z,202409302014_11_Voltage,202409302014_23_X,202409302014_23_Y,202409302014_23_Z,202409302014_23_Voltage
0,0.000000,-0.018021,-0.000935,-0.020087,0.000443,0.006437,-0.004312,0.003345,0.000124,-0.001781,...,-0.005533,-0.040971,-0.016005,-0.001897,-0.020615,0.002035,-0.009129,-0.000674,-0.008132,0.111939
1,0.000156,-0.006662,0.001867,-0.003899,0.000124,0.007665,-0.000951,0.006284,0.001398,0.020879,...,-0.033841,-0.020583,-0.010381,-0.002141,0.006880,0.001398,-0.016798,0.000819,-0.001892,0.092825
2,0.000312,0.013649,0.004049,0.018903,0.000124,-0.000910,0.006552,0.000462,0.001080,0.023079,...,-0.029128,0.002991,-0.005601,0.021896,0.034135,0.001717,0.017786,-0.000157,0.023986,0.070208
3,0.000469,0.020219,0.002618,-0.051073,-0.000195,-0.007906,-0.002169,-0.000201,0.001080,0.023308,...,0.047065,0.003310,0.044207,-0.029773,-0.016545,0.001398,0.011756,0.007153,-0.033423,0.048227
4,0.000625,0.023051,0.012546,-0.062373,-0.000832,0.003494,0.001144,-0.019746,0.000761,0.004017,...,-0.001145,0.002991,0.020391,0.029132,0.044562,0.002354,0.009312,-0.005543,-0.027979,0.025609



合併後的資料形狀:
(30000, 193)




> Table大小皆為30000x48、共4份




###3.補缺漏值>>用means>>各別儲存乾淨資料<br>

In [65]:
# 檢查 merged_df 中是否有 NaN 值
print("處理前 NaN 值的數量：")
print(merged_df.isnull().sum().sum())

# 使用每個欄位的平均值填充 NaN 值，'X_Value' 列通常不需要填充因為它是時間軸，但為確保一致性，可以檢查
# 只對非 'X_Value' 的數據列進行填充
data_cols = [col for col in merged_df.columns if col != 'X_Value']
for col in data_cols:
    if merged_df[col].isnull().any():
        merged_df[col] = merged_df[col].fillna(merged_df[col].mean())

# 再次檢查 NaN 值以確保所有缺漏值都已被處理
print("\n處理後 NaN 值的數量：")
print(merged_df.isnull().sum().sum())

print("\n處理後的資料前5行：")
display(merged_df.head())

print("\n處理後的資料形狀：")
print(merged_df.shape)

# 由於原始需求提到「各別儲存乾淨資料」，這裡假設是指儲存這個完整的、已填充缺漏值的 merged_df
# 如果需要將不同測量類型（如X, Y, Z, Voltage）分開儲存，請進一步說明。
# 我們可以將處理後的資料儲存為一個新的CSV檔案，方便後續使用。
output_file_path = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy_merged_cleaned.csv'
merged_df.to_csv(output_file_path, index=False)
print(f"\n乾淨的合併資料已儲存至：{output_file_path}")

處理前 NaN 值的數量：
0

處理後 NaN 值的數量：
0

處理後的資料前5行：


,X_Value,202409302014_08_X,202409302014_08_Y,202409302014_08_Z,202409302014_08_Voltage,202409302014_02_X,202409302014_02_Y,202409302014_02_Z,202409302014_02_Voltage,202409302014_31_X,...,202409302014_17_Z,202409302014_17_Voltage,202409302014_11_X,202409302014_11_Y,202409302014_11_Z,202409302014_11_Voltage,202409302014_23_X,202409302014_23_Y,202409302014_23_Z,202409302014_23_Voltage
0,0.000000,-0.018021,-0.000935,-0.020087,0.000443,0.006437,-0.004312,0.003345,0.000124,-0.001781,...,-0.005533,-0.040971,-0.016005,-0.001897,-0.020615,0.002035,-0.009129,-0.000674,-0.008132,0.111939
1,0.000156,-0.006662,0.001867,-0.003899,0.000124,0.007665,-0.000951,0.006284,0.001398,0.020879,...,-0.033841,-0.020583,-0.010381,-0.002141,0.006880,0.001398,-0.016798,0.000819,-0.001892,0.092825
2,0.000312,0.013649,0.004049,0.018903,0.000124,-0.000910,0.006552,0.000462,0.001080,0.023079,...,-0.029128,0.002991,-0.005601,0.021896,0.034135,0.001717,0.017786,-0.000157,0.023986,0.070208
3,0.000469,0.020219,0.002618,-0.051073,-0.000195,-0.007906,-0.002169,-0.000201,0.001080,0.023308,...,0.047065,0.003310,0.044207,-0.029773,-0.016545,0.001398,0.011756,0.007153,-0.033423,0.048227
4,0.000625,0.023051,0.012546,-0.062373,-0.000832,0.003494,0.001144,-0.019746,0.000761,0.004017,...,-0.001145,0.002991,0.020391,0.029132,0.044562,0.002354,0.009312,-0.005543,-0.027979,0.025609



處理後的資料形狀：
(30000, 193)

乾淨的合併資料已儲存至：/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/healthy_merged_cleaned.csv


###**去除X_Value

In [66]:
import os
import pandas as pd

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'

# 創建輸出目錄如果它不存在
os.makedirs(output_base_dir, exist_ok=True)

print(f"正在將資料按測量類型儲存到: {output_base_dir}")

# 定義要分開儲存的測量類型
measurement_types = ['X', 'Y', 'Z', 'Voltage']

# 確保 merged_df 已經存在且被處理過 (來自之前的步驟)
if 'merged_df' not in locals() or merged_df.empty:
    print("錯誤: merged_df 不存在或為空，請確認之前的步驟已成功執行。")
else:
    for m_type in measurement_types:
        # 找出所有屬於當前測量類型的欄位
        # 並將這些欄位排序，以便輸出時順序一致
        data_cols_for_type = sorted([col for col in merged_df.columns if col.endswith(f'_{m_type}')])

        # 這裡不包含 'X_Value'，因為使用者要求最終儲存的 CSV 檔案中不包含此欄位
        cols_to_save = data_cols_for_type

        # 創建一個新的DataFrame，只包含當前測量類型的所有數據 (不含 X_Value)
        df_m_type = merged_df[cols_to_save].copy()

        # 定義輸出檔案路徑，使用新的命名格式
        output_file_name = f'Healthy_Merge_Clean_Table_{m_type}.csv'
        output_file_path = os.path.join(output_base_dir, output_file_name)

        # 儲存為CSV檔案
        df_m_type.to_csv(output_file_path, index=False)
        print(f"已儲存 {m_type} 類型資料到: {output_file_path} (形狀: {df_m_type.shape})")

print("所有測量類型的資料已分開儲存完畢。")

正在將資料按測量類型儲存到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data
已儲存 X 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_X.csv (形狀: (30000, 48))
已儲存 Y 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_Y.csv (形狀: (30000, 48))
已儲存 Z 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_Z.csv (形狀: (30000, 48))
已儲存 Voltage 類型資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Healthy_Merge_Clean_Table_Voltage.csv (形狀: (30000, 48))
所有測量類型的資料已分開儲存完畢。


###**Face_Check

In [67]:
import pandas as pd
import os

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'
measurement_types = ['X', 'Y', 'Z', 'Voltage']

print("正在載入並顯示每個分開儲存的 CSV 檔案的前5行和後5行：\n")

for m_type in measurement_types:
    output_file_name = f'Healthy_Merge_Clean_Table_{m_type}.csv'
    output_file_path = os.path.join(output_base_dir, output_file_name)

    if os.path.exists(output_file_path):
        print(f"--- {output_file_name} ---")
        df_loaded = pd.read_csv(output_file_path)
        print("前5行：")
        display(df_loaded.head())
        print("後5行：")
        display(df_loaded.tail())
        print("\n")
    else:
        print(f"錯誤：檔案 {output_file_path} 不存在。")

print("所有檔案檢查完畢。")

正在載入並顯示每個分開儲存的 CSV 檔案的前5行和後5行：

--- Healthy_Merge_Clean_Table_X.csv ---
前5行：


,202409302014_01_X,202409302014_02_X,202409302014_03_X,202409302014_04_X,202409302014_05_X,202409302014_06_X,202409302014_07_X,202409302014_08_X,202409302014_09_X,202409302014_10_X,...,202409302014_39_X,202409302014_40_X,202409302014_41_X,202409302014_42_X,202409302014_43_X,202409302014_44_X,202409302014_45_X,202409302014_46_X,202409302014_47_X,202409302014_48_X
0,0.053857,0.006437,-0.012931,-0.025556,-0.011177,-0.029158,0.024948,-0.018021,-0.018625,0.004698,...,-0.018339,-0.018820,-0.003166,0.019873,0.027487,-0.029371,-0.003316,0.017632,-0.011159,-0.004920
1,0.032555,0.007665,0.011685,0.016332,-0.015793,-0.051850,0.021355,-0.006662,-0.032508,0.015592,...,-0.003302,-0.007441,-0.001389,0.000500,0.005066,-0.009990,-0.029379,0.000773,-0.005031,-0.018377
2,0.004798,-0.000910,0.023469,-0.021005,-0.021959,-0.028541,-0.020189,0.013649,-0.025893,0.017264,...,0.015708,-0.007477,-0.000003,-0.011413,0.006435,0.012263,-0.014929,-0.011990,0.006876,-0.030491
3,-0.019530,-0.007906,-0.002403,-0.030216,-0.002180,0.002617,-0.017600,0.020219,-0.029153,0.020154,...,-0.009273,-0.001912,-0.042673,0.000390,-0.014527,0.027033,0.008128,-0.028621,0.003613,-0.006652
4,-0.004428,0.003494,-0.023142,-0.033537,0.027174,0.025222,-0.006737,0.023051,-0.017705,0.027609,...,-0.020363,-0.010665,-0.037857,-0.003853,-0.016170,0.007323,0.038054,-0.014440,-0.015312,-0.007826


後5行：


,202409302014_01_X,202409302014_02_X,202409302014_03_X,202409302014_04_X,202409302014_05_X,202409302014_06_X,202409302014_07_X,202409302014_08_X,202409302014_09_X,202409302014_10_X,...,202409302014_39_X,202409302014_40_X,202409302014_41_X,202409302014_42_X,202409302014_43_X,202409302014_44_X,202409302014_45_X,202409302014_46_X,202409302014_47_X,202409302014_48_X
29995,0.009914,0.014185,-0.029351,0.031321,0.011528,0.039559,0.009563,0.050527,0.024373,-0.005044,...,0.016484,-0.003026,0.006194,-0.029694,-0.012847,0.010255,-0.008182,0.012265,0.011893,-0.010491
29996,0.006148,0.009509,0.060023,0.026032,0.022297,0.024422,0.004305,0.038846,0.026338,-0.022072,...,0.029507,0.015866,-0.026538,-0.031725,0.005595,-0.007568,0.002320,0.033503,0.002971,-0.010445
29997,0.003009,0.027948,0.026799,-0.003476,0.014452,-0.014357,0.004005,0.025595,-0.018106,-0.007075,...,0.037716,0.028303,-0.017614,-0.010021,0.009148,-0.016398,0.017311,0.013075,0.000870,-0.031694
29998,-0.010613,0.015428,0.034973,-0.004707,-0.004110,-0.011784,0.012020,0.007027,-0.040779,-0.002027,...,0.015222,0.021730,-0.004587,0.003046,0.009030,0.000644,0.009845,0.001342,-0.007462,-0.020823
29999,-0.000942,-0.019569,-0.013964,-0.026634,-0.008798,-0.005263,-0.009914,-0.022176,-0.019426,-0.002655,...,-0.002495,0.005518,0.037897,0.037305,0.007367,0.007717,0.018241,-0.006729,0.010359,0.021872




--- Healthy_Merge_Clean_Table_Y.csv ---
前5行：


...



--- Healthy_Merge_Clean_Table_Voltage.csv ---
前5行：


,202409302014_01_Voltage,202409302014_02_Voltage,202409302014_03_Voltage,202409302014_04_Voltage,202409302014_05_Voltage,202409302014_06_Voltage,202409302014_07_Voltage,202409302014_08_Voltage,202409302014_09_Voltage,202409302014_10_Voltage,...,202409302014_39_Voltage,202409302014_40_Voltage,202409302014_41_Voltage,202409302014_42_Voltage,202409302014_43_Voltage,202409302014_44_Voltage,202409302014_45_Voltage,202409302014_46_Voltage,202409302014_47_Voltage,202409302014_48_Voltage
0,0.000443,0.000124,0.000761,0.002672,0.000443,0.001080,0.002354,0.000443,-0.057536,0.001080,...,-0.021220,0.000761,0.000761,0.003310,0.000761,-0.078242,0.001398,-0.000513,0.002672,0.025290
1,0.002672,0.001398,0.001080,0.002991,0.046953,0.001398,0.002991,0.000124,-0.037785,0.029113,...,0.003310,0.001398,0.001398,0.003628,0.000124,-0.100860,0.001398,0.000124,0.002672,-0.000513
2,0.002672,0.001080,0.001398,0.003628,0.092188,0.001398,0.002991,0.000124,-0.015167,0.082632,...,0.002672,0.001080,0.000761,0.002991,0.000443,-0.106276,0.002035,0.000761,0.002672,0.000761
3,0.002991,0.001080,0.001080,0.002991,0.114488,0.002672,0.003310,-0.000195,0.002991,0.110665,...,0.002991,0.002672,0.002035,0.002672,0.000443,-0.102453,0.000761,0.001080,0.002991,0.000443
4,0.002991,0.000761,0.001080,0.003310,0.125637,0.002035,0.002672,-0.000832,0.002672,0.120859,...,0.002672,0.071163,0.001398,0.003628,0.000124,-0.090029,0.001080,0.002354,0.003947,0.001398


後5行：


,202409302014_01_Voltage,202409302014_02_Voltage,202409302014_03_Voltage,202409302014_04_Voltage,202409302014_05_Voltage,202409302014_06_Voltage,202409302014_07_Voltage,202409302014_08_Voltage,202409302014_09_Voltage,202409302014_10_Voltage,...,202409302014_39_Voltage,202409302014_40_Voltage,202409302014_41_Voltage,202409302014_42_Voltage,202409302014_43_Voltage,202409302014_44_Voltage,202409302014_45_Voltage,202409302014_46_Voltage,202409302014_47_Voltage,202409302014_48_Voltage
29995,0.001717,0.001080,-0.049572,0.000443,0.001717,0.003310,0.001080,-0.093852,0.000443,0.001080,...,0.001717,0.000124,0.002991,0.000124,0.002672,0.002035,0.000124,0.002991,0.122770,0.002672
29996,0.002035,-0.000513,-0.023131,0.001717,0.001080,0.002672,0.000124,-0.101179,0.001398,0.001398,...,0.000124,0.001080,0.003628,0.000124,0.002991,0.001717,-0.000513,0.002672,0.110346,0.001717
29997,0.001398,0.000761,0.000761,0.000761,0.001717,0.002991,0.000124,-0.104046,0.001717,0.001080,...,0.000761,0.001080,0.003310,0.000761,0.002991,0.002035,0.000124,0.002991,0.092507,0.002672
29998,-0.000513,0.001398,0.003310,0.000443,0.001080,0.002991,0.000443,-0.094808,0.001398,0.001398,...,0.001080,0.001717,0.002991,0.000761,0.001717,0.001717,-0.000513,0.003310,0.074030,0.002991
29999,0.001080,0.001080,0.003310,0.001398,0.001080,0.003947,0.000443,-0.077287,0.000443,0.002035,...,0.002035,0.001398,0.003628,0.001398,-0.034281,0.001398,-0.000195,0.002672,0.053005,-0.014530




所有檔案檢查完畢。


##Fauty

###1.讀檔>>去除取樣資料的檔頭>>取22~30023

In [68]:
import os
import pandas as pd
import numpy as np

# 重複使用先前定義的函數來尋找資料標題行索引
def get_data_header_line_index(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    header_end_count = 0
    data_header_line_index = -1 # 資料實際標題行的0索引行號
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_end_count += 1
        if header_end_count == 2: # 在第二個 '***End_of_Header***' 之後
            # 尋找包含資料欄位名稱的行
            # 使用 'X_Value\tX\tY\tZ\tVoltage\tComment' 作為資料標題行的唯一識別符號
            if 'X_Value\tX\tY\tZ\tVoltage\tComment' in line:
                data_header_line_index = i
                break
            # 處理檔頭可能在 End_of_Header 後下一行的情況
            elif i + 1 < len(lines) and lines[i+1].startswith('X_Value\tX\tY\tZ\tVoltage\tComment'):
                 data_header_line_index = i + 1
                 break
    if data_header_line_index == -1:
        raise ValueError(f"在檔案中找不到資料標題行: {file_path}")
    return data_header_line_index

###2.合併資料>>Healthy的各維度>>各別組成一張Table

In [69]:
# --- 步驟 1: 載入並合併故障資料 ---
print("--- 步驟 1: 載入並合併故障資料 ---")

# 定義包含故障 LVM 檔案的目錄
faulty_data_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/faulty/'
lvm_files_faulty = [os.path.join(faulty_data_dir, f) for f in os.listdir(faulty_data_dir) if f.endswith('.lvm')]

all_processed_dfs_faulty = []

print(f"在目錄中找到 {len(lvm_files_faulty)} 個故障 LVM 檔案: {faulty_data_dir}\n")

for file_path in lvm_files_faulty:
    try:
        data_header_0_indexed = get_data_header_line_index(file_path)
        skiprows_for_pd_read_csv = data_header_0_indexed + 1

        df_temp = pd.read_csv(
            file_path,
            skiprows=skiprows_for_pd_read_csv,
            sep='\t',
            header=None
        )

        with open(file_path, 'r', encoding='utf-8') as f:
            header_line_content = f.readlines()[data_header_0_indexed]
        columns = [col.strip() for col in header_line_content.split('\t') if col.strip()]

        if 'Comment' in columns:
            columns.remove('Comment')

        if len(df_temp.columns) == len(columns):
            df_temp.columns = columns
        else:
            print(f"警告: {file_path} 中的欄位計數不匹配。預期 {len(columns)} 個欄位，但 DataFrame 有 {len(df_temp.columns)} 個。跳過此檔案。\n")
            continue

        filename_without_ext = os.path.basename(file_path).split('.')[0]
        renamed_cols = {col: f"{filename_without_ext}_{col}" for col in columns if col != 'X_Value'}
        df_temp = df_temp.rename(columns=renamed_cols)

        all_processed_dfs_faulty.append(df_temp)

    except Exception as e:
        print(f"處理 {file_path} 時發生錯誤: {e}\n")

faulty_merged_df = pd.DataFrame()
if all_processed_dfs_faulty:
    faulty_merged_df = all_processed_dfs_faulty[0]
    for i in range(1, len(all_processed_dfs_faulty)):
        faulty_merged_df = pd.merge(faulty_merged_df, all_processed_dfs_faulty[i], on='X_Value', how='outer')

    faulty_merged_df = faulty_merged_df.sort_values(by='X_Value').reset_index(drop=True)

    print("\n所有故障檔案處理和合併成功！")
    print("故障合併資料 (前5行):")
    display(faulty_merged_df.head())
    print("\n故障合併資料形狀:")
    print(faulty_merged_df.shape)
else:
    print("沒有成功處理或找到要合併的故障 DataFrames。")
    exit("沒有故障資料要處理。")

# 以下程式碼已移出此 Cell

--- 步驟 1: 載入並合併故障資料 ---
在目錄中找到 48 個故障 LVM 檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Training/faulty/


所有故障檔案處理和合併成功！
故障合併資料 (前5行):


,X_Value,202409302025_19_X,202409302025_19_Y,202409302025_19_Z,202409302025_19_Voltage,202409302025_24_X,202409302025_24_Y,202409302025_24_Z,202409302025_24_Voltage,202409302025_08_X,...,202409302025_43_Z,202409302025_43_Voltage,202409302025_42_X,202409302025_42_Y,202409302025_42_Z,202409302025_42_Voltage,202409302025_32_X,202409302025_32_Y,202409302025_32_Z,202409302025_32_Voltage
0,0.000000,0.012486,0.015481,0.072605,0.003947,0.002503,0.005720,0.004655,0.002991,0.005649,...,0.014700,0.003947,-0.000176,0.006084,0.004248,0.002035,-0.025835,0.005890,-0.009668,-0.056899
1,0.000156,0.034577,-0.015628,0.045438,0.004265,0.024006,-0.018961,0.010187,0.002991,0.011834,...,0.033275,0.002991,-0.020287,-0.000792,-0.006387,0.000761,-0.116303,-0.009691,0.047929,-0.084932
2,0.000312,0.043336,0.008064,0.035235,0.003628,0.035032,-0.002937,0.009234,0.002035,0.007815,...,0.010894,0.003947,-0.020585,0.004072,-0.003384,0.001398,-0.022613,0.010335,0.012665,-0.103409
3,0.000469,0.036487,-0.004241,0.024058,0.003947,0.028630,-0.006635,-0.011620,0.002991,-0.015432,...,0.033379,0.003947,-0.015786,-0.000959,0.004226,0.003947,0.049476,-0.017038,-0.013879,-0.107869
4,0.000625,-0.007239,-0.004633,0.045696,0.004584,-0.006546,-0.000451,0.007975,0.001717,0.013059,...,0.005378,0.003628,-0.012530,-0.010563,-0.008538,0.002672,0.104103,0.021635,0.003678,-0.097356



故障合併資料形狀:
(30000, 193)


###3.補缺漏值>>用means>>各別儲存乾淨資料<br>

In [70]:
# --- 步驟 2: 處理故障資料中的缺漏值 ---
print("\n--- 步驟 2: 處理故障資料中的缺漏值 ---")

print(f"故障資料: 填充前 NaN 值的數量: {faulty_merged_df.isnull().sum().sum()}")

data_cols_faulty = [col for col in faulty_merged_df.columns if col != 'X_Value']
for col in data_cols_faulty:
    if faulty_merged_df[col].isnull().any():
        faulty_merged_df[col] = faulty_merged_df[col].fillna(faulty_merged_df[col].mean())

print(f"故障資料: 填充後 NaN 值的數量: {faulty_merged_df.isnull().sum().sum()}")

print("\n處理 NaN 後的故障合併資料 (前5行):")
display(faulty_merged_df.head())

print("\n處理 NaN 後的故障合併資料形狀:")
print(faulty_merged_df.shape)


--- 步驟 2: 處理故障資料中的缺漏值 ---
故障資料: 填充前 NaN 值的數量: 0
故障資料: 填充後 NaN 值的數量: 0

處理 NaN 後的故障合併資料 (前5行):


,X_Value,202409302025_19_X,202409302025_19_Y,202409302025_19_Z,202409302025_19_Voltage,202409302025_24_X,202409302025_24_Y,202409302025_24_Z,202409302025_24_Voltage,202409302025_08_X,...,202409302025_43_Z,202409302025_43_Voltage,202409302025_42_X,202409302025_42_Y,202409302025_42_Z,202409302025_42_Voltage,202409302025_32_X,202409302025_32_Y,202409302025_32_Z,202409302025_32_Voltage
0,0.000000,0.012486,0.015481,0.072605,0.003947,0.002503,0.005720,0.004655,0.002991,0.005649,...,0.014700,0.003947,-0.000176,0.006084,0.004248,0.002035,-0.025835,0.005890,-0.009668,-0.056899
1,0.000156,0.034577,-0.015628,0.045438,0.004265,0.024006,-0.018961,0.010187,0.002991,0.011834,...,0.033275,0.002991,-0.020287,-0.000792,-0.006387,0.000761,-0.116303,-0.009691,0.047929,-0.084932
2,0.000312,0.043336,0.008064,0.035235,0.003628,0.035032,-0.002937,0.009234,0.002035,0.007815,...,0.010894,0.003947,-0.020585,0.004072,-0.003384,0.001398,-0.022613,0.010335,0.012665,-0.103409
3,0.000469,0.036487,-0.004241,0.024058,0.003947,0.028630,-0.006635,-0.011620,0.002991,-0.015432,...,0.033379,0.003947,-0.015786,-0.000959,0.004226,0.003947,0.049476,-0.017038,-0.013879,-0.107869
4,0.000625,-0.007239,-0.004633,0.045696,0.004584,-0.006546,-0.000451,0.007975,0.001717,0.013059,...,0.005378,0.003628,-0.012530,-0.010563,-0.008538,0.002672,0.104103,0.021635,0.003678,-0.097356



處理 NaN 後的故障合併資料形狀:
(30000, 193)


###**去除X_Value

In [71]:
# --- 步驟 3: 分離並儲存乾淨的故障資料 ---
print("\n--- 步驟 3: 分離並儲存乾淨的故障資料 ---")

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'
os.makedirs(output_base_dir, exist_ok=True)

print(f"正在將故障資料按測量類型儲存到: {output_base_dir}")

measurement_types = ['X', 'Y', 'Z', 'Voltage']

for m_type in measurement_types:
    data_cols_for_type = sorted([col for col in faulty_merged_df.columns if col.endswith(f'_{m_type}')])
    df_m_type = faulty_merged_df[data_cols_for_type].copy()

    output_file_name = f'Faulty_Merge_Clean_Table_{m_type}.csv'
    output_file_path = os.path.join(output_base_dir, output_file_name)

    df_m_type.to_csv(output_file_path, index=False)
    print(f"已儲存 {m_type} 故障資料到: {output_file_path} (形狀: {df_m_type.shape})")

print("\n所有故障測量類型資料已分離並儲存完畢。")


--- 步驟 3: 分離並儲存乾淨的故障資料 ---
正在將故障資料按測量類型儲存到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data
已儲存 X 故障資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Faulty_Merge_Clean_Table_X.csv (形狀: (30000, 48))
已儲存 Y 故障資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Faulty_Merge_Clean_Table_Y.csv (形狀: (30000, 48))
已儲存 Z 故障資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Faulty_Merge_Clean_Table_Z.csv (形狀: (30000, 48))
已儲存 Voltage 故障資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Faulty_Merge_Clean_Table_Voltage.csv (形狀: (30000, 48))

所有故障測量類型資料已分離並儲存完畢。


###**Face_Check

In [73]:
# --- 步驟 4: 驗證儲存的故障資料 ---
print("\n--- 步驟 4: 驗證儲存的故障資料 ---")

print("正在載入並顯示每個儲存的故障 CSV 檔案的前5行和後5行：\n")

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'
measurement_types = ['X', 'Y', 'Z', 'Voltage']

for m_type in measurement_types:
    output_file_name = f'Faulty_Merge_Clean_Table_{m_type}.csv'
    output_file_path = os.path.join(output_base_dir, output_file_name)

    if os.path.exists(output_file_path):
        print(f"--- {output_file_name} ---")
        df_loaded_faulty = pd.read_csv(output_file_path)
        print("前5行：")
        display(df_loaded_faulty.head())
        print("後5行：")
        display(df_loaded_faulty.tail())
        print("\n")
    else:
        print(f"錯誤: 檔案 {output_file_path} 不存在。")

print("所有故障檔案檢查成功。")


--- 步驟 4: 驗證儲存的故障資料 ---
正在載入並顯示每個儲存的故障 CSV 檔案的前5行和後5行：

--- Faulty_Merge_Clean_Table_X.csv ---
前5行：


,202409302025_01_X,202409302025_02_X,202409302025_03_X,202409302025_04_X,202409302025_05_X,202409302025_06_X,202409302025_07_X,202409302025_08_X,202409302025_09_X,202409302025_10_X,...,202409302025_39_X,202409302025_40_X,202409302025_41_X,202409302025_42_X,202409302025_43_X,202409302025_44_X,202409302025_45_X,202409302025_46_X,202409302025_47_X,202409302025_48_X
0,0.012858,0.002999,-0.010269,0.006186,-0.023477,0.028791,0.010270,0.005649,0.030702,-0.026971,...,0.012274,0.105804,-0.006244,-0.000176,0.005946,0.006647,-0.021958,-0.015788,-0.011093,-0.008140
1,-0.032346,0.024062,0.025824,0.006513,-0.016347,-0.007647,0.026750,0.011834,0.013707,0.013238,...,0.018455,0.009045,-0.004631,-0.020287,-0.007058,0.016782,-0.001428,-0.009923,-0.010077,0.012112
2,-0.011552,-0.004301,0.048699,0.018335,-0.000228,-0.005740,-0.004280,0.007815,0.001053,0.047169,...,-0.004505,-0.029644,0.023080,-0.020585,0.011544,-0.008638,-0.018401,-0.011965,0.001947,-0.004476
3,0.029130,-0.001112,0.013614,-0.000970,0.022173,0.023577,0.031438,-0.015432,0.009472,0.040392,...,-0.034062,-0.005381,-0.013912,-0.015786,0.015199,-0.026361,-0.023552,-0.007284,0.001212,-0.009097
4,0.068891,0.006404,-0.015303,0.009529,0.042601,-0.003908,0.028108,0.013059,0.012413,0.038745,...,-0.027097,0.009454,-0.040518,-0.012530,0.015102,-0.022047,0.005120,-0.008506,-0.024614,-0.019782


後5行：


,202409302025_01_X,202409302025_02_X,202409302025_03_X,202409302025_04_X,202409302025_05_X,202409302025_06_X,202409302025_07_X,202409302025_08_X,202409302025_09_X,202409302025_10_X,...,202409302025_39_X,202409302025_40_X,202409302025_41_X,202409302025_42_X,202409302025_43_X,202409302025_44_X,202409302025_45_X,202409302025_46_X,202409302025_47_X,202409302025_48_X
29995,-0.000412,-0.010485,-0.007288,-0.005126,-0.036266,0.011777,0.009521,0.026941,0.003418,0.004142,...,0.029461,-0.010297,0.001634,-0.003964,-0.021346,0.015271,0.006734,-0.016877,0.013114,0.007372
29996,0.024004,-0.009388,0.002636,-0.033131,-0.014012,0.009749,0.015971,0.028071,-0.033309,-0.011212,...,-0.033256,-0.008020,-0.004931,-0.003562,0.022215,0.044101,0.008128,0.025746,0.005492,0.014650
29997,-0.000585,-0.007943,0.006365,-0.002439,0.003525,-0.018827,-0.010121,-0.009411,-0.039153,0.013690,...,-0.108616,0.014722,-0.001619,-0.003147,0.046216,0.016757,0.018534,0.012399,-0.014988,-0.010553
29998,-0.005939,-0.013019,-0.005654,0.010575,0.014369,-0.027618,-0.019645,0.003260,-0.045761,0.009351,...,-0.025787,0.021654,0.018252,0.015419,0.019578,-0.002226,0.010844,0.020933,-0.024483,0.008587
29999,0.003507,-0.017311,-0.008227,0.004535,0.033406,-0.009325,-0.011984,0.013788,-0.024657,0.005681,...,0.040716,0.013243,0.023685,0.010397,0.008243,-0.000824,0.006122,0.001355,-0.014398,-0.009326




--- Faulty_Merge_Clean_Table_Y.csv ---
前5行：


...



--- Faulty_Merge_Clean_Table_Voltage.csv ---
前5行：


,202409302025_01_Voltage,202409302025_02_Voltage,202409302025_03_Voltage,202409302025_04_Voltage,202409302025_05_Voltage,202409302025_06_Voltage,202409302025_07_Voltage,202409302025_08_Voltage,202409302025_09_Voltage,202409302025_10_Voltage,...,202409302025_39_Voltage,202409302025_40_Voltage,202409302025_41_Voltage,202409302025_42_Voltage,202409302025_43_Voltage,202409302025_44_Voltage,202409302025_45_Voltage,202409302025_46_Voltage,202409302025_47_Voltage,202409302025_48_Voltage
0,0.003310,0.001717,-0.106594,0.002354,0.001398,0.003628,0.120540,0.003310,0.003628,0.001080,...,0.002672,-0.094489,0.002035,0.002035,0.003947,0.001398,-0.036829,0.002991,0.001398,0.003628
1,0.003310,0.001717,-0.104364,0.001717,0.001717,0.003947,0.107161,0.003310,0.004265,0.000761,...,0.001717,-0.081109,0.002991,0.000761,0.002991,0.000761,-0.011981,0.002035,0.001717,0.003628
2,0.003628,0.001717,-0.093533,0.002354,0.002672,0.005858,0.095374,0.004265,0.004584,0.001398,...,0.000761,-0.061359,0.001717,0.001398,0.003947,0.000124,0.004265,0.001717,0.002354,0.002672
3,0.004584,0.001398,-0.077924,0.001717,0.001717,0.004584,0.072119,0.002991,0.002991,0.001717,...,0.001398,-0.041926,0.001080,0.003947,0.003947,0.001080,0.003947,0.001398,0.002991,0.003310
4,0.003947,0.001398,-0.059129,0.003310,0.002672,0.004265,0.049183,0.002991,0.003628,0.001398,...,0.000761,-0.017078,0.001717,0.002672,0.003628,0.001080,0.003628,0.001717,0.003310,0.003628


後5行：


,202409302025_01_Voltage,202409302025_02_Voltage,202409302025_03_Voltage,202409302025_04_Voltage,202409302025_05_Voltage,202409302025_06_Voltage,202409302025_07_Voltage,202409302025_08_Voltage,202409302025_09_Voltage,202409302025_10_Voltage,...,202409302025_39_Voltage,202409302025_40_Voltage,202409302025_41_Voltage,202409302025_42_Voltage,202409302025_43_Voltage,202409302025_44_Voltage,202409302025_45_Voltage,202409302025_46_Voltage,202409302025_47_Voltage,202409302025_48_Voltage
29995,0.001717,0.003310,0.002672,0.000761,0.004584,0.013185,0.002991,0.003947,0.001080,0.003947,...,-0.025361,0.002991,0.001398,0.003947,0.001398,-0.100542,0.001717,0.002035,0.003947,0.000443
29996,0.001717,0.003628,0.002354,0.001717,0.003947,0.074030,0.002354,0.003628,0.001398,0.002991,...,-0.071552,0.003628,0.002354,0.003947,0.001717,-0.102134,0.002672,0.002672,0.004584,0.001398
29997,0.001398,-0.026635,0.001398,0.001080,0.003310,0.103657,0.002991,0.004265,0.002035,0.003628,...,-0.092578,0.003310,0.001717,0.004265,0.001080,-0.091622,0.002354,0.002354,0.002991,0.000443
29998,0.001080,-0.071234,0.002991,0.001717,0.004265,0.119903,0.002672,0.004265,0.001717,0.004265,...,-0.105639,0.002672,0.002991,0.004265,0.000443,-0.074420,0.002035,0.002991,0.003628,0.001080
29999,0.001717,-0.093852,0.002991,0.000443,0.003947,0.127549,0.002991,0.003628,0.001717,-0.045749,...,-0.105957,0.002035,0.002991,0.004265,0.000761,-0.054987,0.002991,0.001717,0.003628,0.001398




所有故障檔案檢查成功。


##Testing

# Task
載入並合併 `/content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing` 路徑下的所有 LVM 檔案。每個檔案將依據標頭資訊被讀取，將 `X_Value` 作為共同鍵，其他維度（X, Y, Z, Voltage）的欄位會加上檔案名稱前綴，最後合併成一個名為 `testing_merged_df` 的 DataFrame。

## 載入並合併測試資料

### Subtask:
載入 /content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing 路徑下的所有 LVM 檔案。每個檔案將依據標頭資訊被讀取，將 `X_Value` 作為共同鍵，其他維度（X, Y, Z, Voltage）的欄位會加上檔案名稱前綴，最後合併成一個名為 `testing_merged_df` 的 DataFrame。


**Reasoning**:
The subtask requires loading, processing, and merging all LVM files from the specified testing directory into a single DataFrame, similar to how healthy and faulty data were handled previously. This code block will perform these steps, including identifying the header, reading data, renaming columns with file prefixes, and merging them using 'X_Value' as the common key.



In [74]:
import os
import pandas as pd
import numpy as np

# The get_data_header_line_index function is already defined in a previous cell.
# Re-define it here for clarity if this cell were to be run independently, but it's available in the kernel state.
# For this interactive session, we assume it's already in scope.

def get_data_header_line_index(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    header_end_count = 0
    data_header_line_index = -1 # 資料實際標題行的0索引行號
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_end_count += 1
        if header_end_count == 2: # 在第二個 '***End_of_Header***' 之後
            if 'X_Value\tX\tY\tZ\tVoltage\tComment' in line:
                data_header_line_index = i
                break
            elif i + 1 < len(lines) and lines[i+1].startswith('X_Value\tX\tY\tZ\tVoltage\tComment'):
                 data_header_line_index = i + 1
                 break
    if data_header_line_index == -1:
        raise ValueError(f"在檔案中找不到資料標題行: {file_path}")
    return data_header_line_index


# --- 步驟 1: 載入並合併測試資料 ---
print("--- 步驟 1: 載入並合併測試資料 ---")

# 1. 定義測試資料的目錄路徑
testing_data_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing/'

# 2. 獲取該目錄下所有副檔名為 `.lvm` 的檔案清單
lvm_files_testing = [os.path.join(testing_data_dir, f) for f in os.listdir(testing_data_dir) if f.endswith('.lvm')]

# 3. 初始化一個空列表用於存放每個 LVM 檔案處理後的 DataFrame
all_processed_dfs_testing = []

print(f"在目錄中找到 {len(lvm_files_testing)} 個測試 LVM 檔案: {testing_data_dir}\n")

# 4. 遍歷每個 LVM 檔案
for file_path in lvm_files_testing:
    try:
        # a. 使用 get_data_header_line_index() 函數來確定資料標頭所在的行。
        data_header_0_indexed = get_data_header_line_index(file_path)

        # b. 計算 pandas.read_csv 需要跳過的行數
        skiprows_for_pd_read_csv = data_header_0_indexed + 1

        # c. 使用 pandas.read_csv() 讀取檔案
        df_temp = pd.read_csv(
            file_path,
            skiprows=skiprows_for_pd_read_csv,
            sep='\t',
            header=None # 我們將手動指定欄位名稱
        )

        # d. 從資料標頭行內容中提取並清理實際的欄位名稱
        with open(file_path, 'r', encoding='utf-8') as f:
            header_line_content = f.readlines()[data_header_0_indexed]
        columns = [col.strip() for col in header_line_content.split('\t') if col.strip()]

        # e. 如果存在 'Comment' 欄位，則從欄位名稱列表中移除它
        if 'Comment' in columns:
            columns.remove('Comment')

        # f. 將清理後的欄位名稱指定給讀取到的 DataFrame。
        if len(df_temp.columns) == len(columns):
            df_temp.columns = columns
        else:
            print(f"警告: {file_path} 中的欄位計數不匹配。預期 {len(columns)} 個欄位，但 DataFrame 有 {len(df_temp.columns)} 個。跳過此檔案。\n")
            continue

        # g. 為除 'X_Value' 之外的所有資料欄位，將其重新命名，前綴為檔案名稱（不含副檔名）。
        filename_without_ext = os.path.basename(file_path).split('.')[0]
        renamed_cols = {col: f"{filename_without_ext}_{col}" for col in columns if col != 'X_Value'}
        df_temp = df_temp.rename(columns=renamed_cols)

        # h. 將處理後的 DataFrame 加入 all_processed_dfs_testing 列表。
        all_processed_dfs_testing.append(df_temp)

    except Exception as e:
        print(f"處理 {file_path} 時發生錯誤: {e}\n")

# 5. 初始化 testing_merged_df 為 all_processed_dfs_testing 列表中的第一個 DataFrame。
# 6. 逐一將列表中的其餘 DataFrame 與 testing_merged_df 進行合併
testing_merged_df = pd.DataFrame()
if all_processed_dfs_testing:
    testing_merged_df = all_processed_dfs_testing[0]
    for i in range(1, len(all_processed_dfs_testing)):
        testing_merged_df = pd.merge(testing_merged_df, all_processed_dfs_testing[i], on='X_Value', how='outer')

    # 7. 合併完成後，依照 'X_Value' 欄位對 testing_merged_df 進行排序，並重置索引 (drop=True)。
    testing_merged_df = testing_merged_df.sort_values(by='X_Value').reset_index(drop=True)

    print("\n所有測試檔案處理和合併成功！")
    print("測試合併資料 (前5行):")
    display(testing_merged_df.head())
    print("\n測試合併資料形狀:")
    print(testing_merged_df.shape)
else:
    print("沒有成功處理或找到要合併的測試 DataFrames。")
    raise ValueError("沒有測試資料要處理。")


--- 步驟 1: 載入並合併測試資料 ---
在目錄中找到 24 個測試 LVM 檔案: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing/


所有測試檔案處理和合併成功！
測試合併資料 (前5行):


,X_Value,healthy_ (2)_X,healthy_ (2)_Y,healthy_ (2)_Z,healthy_ (2)_Voltage,healthy_ (6)_X,healthy_ (6)_Y,healthy_ (6)_Z,healthy_ (6)_Voltage,faulty_ (2)_X,...,faulty_ (9)_Z,faulty_ (9)_Voltage,faulty_ (7)_X,faulty_ (7)_Y,faulty_ (7)_Z,faulty_ (7)_Voltage,healthy_ (4)_X,healthy_ (4)_Y,healthy_ (4)_Z,healthy_ (4)_Voltage
0,0.000000,-0.045524,-0.003567,0.025410,0.000124,-0.010527,0.005869,0.012485,0.003310,0.025030,...,0.008445,0.004265,-0.059996,-0.038668,0.008892,-0.069323,0.051067,-0.017984,0.047828,-0.079517
1,0.000156,-0.005067,-0.005784,0.016875,0.001080,-0.000990,-0.006282,-0.000036,0.002672,-0.007492,...,0.012725,0.003947,-0.096980,0.040895,0.070954,-0.092896,-0.004804,0.002462,-0.010724,-0.061677
2,0.000312,0.031375,-0.004411,0.005744,0.001398,0.017864,-0.002165,-0.004002,0.002991,-0.025616,...,-0.019567,0.004265,0.011165,-0.020713,-0.022181,-0.104683,0.023639,-0.005880,-0.027477,-0.043200
3,0.000469,0.041163,-0.007327,-0.010855,0.001080,0.017358,-0.002040,0.007462,0.002991,-0.017450,...,0.020769,0.004584,0.065965,0.012673,0.017595,-0.102453,0.017523,0.003188,-0.026131,-0.020264
4,0.000625,-0.009538,-0.004630,-0.021823,0.001717,-0.007930,0.003853,0.023273,0.002991,0.005600,...,0.036752,0.003628,0.096978,-0.003132,-0.013711,-0.091303,0.019182,0.008310,-0.040259,0.002991



測試合併資料形狀:
(30000, 97)


**Reasoning**:
The next step is to handle any missing values in the `testing_merged_df` by filling them with the mean of their respective columns, similar to how missing values were handled for healthy and faulty data.



In [75]:
# --- 步驟 2: 處理測試資料中的缺漏值 ---
print("\n--- 步驟 2: 處理測試資料中的缺漏值 ---")

# 檢查 testing_merged_df 中是否有 NaN 值
print(f"測試資料: 填充前 NaN 值的數量: {testing_merged_df.isnull().sum().sum()}")

# 使用每個欄位的平均值填充 NaN 值，'X_Value' 列通常不需要填充因為它是時間軸，但為確保一致性，可以檢查
# 只對非 'X_Value' 的數據列進行填充
data_cols_testing = [col for col in testing_merged_df.columns if col != 'X_Value']
for col in data_cols_testing:
    if testing_merged_df[col].isnull().any():
        testing_merged_df[col] = testing_merged_df[col].fillna(testing_merged_df[col].mean())

# 再次檢查 NaN 值以確保所有缺漏值都已被處理
print(f"測試資料: 填充後 NaN 值的數量: {testing_merged_df.isnull().sum().sum()}")

print("\n處理 NaN 後的測試合併資料 (前5行):")
display(testing_merged_df.head())

print("\n處理 NaN 後的測試合併資料形狀:")
print(testing_merged_df.shape)

# 由於原始需求提到「各別儲存乾淨資料」，這裡假設是指儲存這個完整的、已填充缺漏值的 testing_merged_df
# 如果需要將不同測量類型（如X, Y, Z, Voltage）分開儲存，請進一步說明。
# 我們可以將處理後的資料儲存為一個新的CSV檔案，方便後續使用。
output_file_path_full = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing/testing_merged_cleaned.csv'
testing_merged_df.to_csv(output_file_path_full, index=False)
print(f"\n乾淨的測試合併資料已儲存至：{output_file_path_full}")


--- 步驟 2: 處理測試資料中的缺漏值 ---
測試資料: 填充前 NaN 值的數量: 0
測試資料: 填充後 NaN 值的數量: 0

處理 NaN 後的測試合併資料 (前5行):


,X_Value,healthy_ (2)_X,healthy_ (2)_Y,healthy_ (2)_Z,healthy_ (2)_Voltage,healthy_ (6)_X,healthy_ (6)_Y,healthy_ (6)_Z,healthy_ (6)_Voltage,faulty_ (2)_X,...,faulty_ (9)_Z,faulty_ (9)_Voltage,faulty_ (7)_X,faulty_ (7)_Y,faulty_ (7)_Z,faulty_ (7)_Voltage,healthy_ (4)_X,healthy_ (4)_Y,healthy_ (4)_Z,healthy_ (4)_Voltage
0,0.000000,-0.045524,-0.003567,0.025410,0.000124,-0.010527,0.005869,0.012485,0.003310,0.025030,...,0.008445,0.004265,-0.059996,-0.038668,0.008892,-0.069323,0.051067,-0.017984,0.047828,-0.079517
1,0.000156,-0.005067,-0.005784,0.016875,0.001080,-0.000990,-0.006282,-0.000036,0.002672,-0.007492,...,0.012725,0.003947,-0.096980,0.040895,0.070954,-0.092896,-0.004804,0.002462,-0.010724,-0.061677
2,0.000312,0.031375,-0.004411,0.005744,0.001398,0.017864,-0.002165,-0.004002,0.002991,-0.025616,...,-0.019567,0.004265,0.011165,-0.020713,-0.022181,-0.104683,0.023639,-0.005880,-0.027477,-0.043200
3,0.000469,0.041163,-0.007327,-0.010855,0.001080,0.017358,-0.002040,0.007462,0.002991,-0.017450,...,0.020769,0.004584,0.065965,0.012673,0.017595,-0.102453,0.017523,0.003188,-0.026131,-0.020264
4,0.000625,-0.009538,-0.004630,-0.021823,0.001717,-0.007930,0.003853,0.023273,0.002991,0.005600,...,0.036752,0.003628,0.096978,-0.003132,-0.013711,-0.091303,0.019182,0.008310,-0.040259,0.002991



處理 NaN 後的測試合併資料形狀:
(30000, 97)

乾淨的測試合併資料已儲存至：/content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing/testing_merged_cleaned.csv


**Reasoning**:
The previous steps successfully loaded, merged, and cleaned the testing data, including handling missing values and saving the complete `testing_merged_df`. Following the pattern established for healthy and faulty data, the next logical step is to separate the `testing_merged_df` into individual CSV files for each measurement type (X, Y, Z, Voltage), excluding the `X_Value` column, and save them to the specified clean data directory. This prepares the data for further analysis or model training by providing segregated datasets for each dimension.



In [76]:
import os
import pandas as pd

# --- 步驟 3: 分離並儲存乾淨的測試資料 (去除X_Value) ---
print("\n--- 步驟 3: 分離並儲存乾淨的測試資料 (去除X_Value) ---")

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'
os.makedirs(output_base_dir, exist_ok=True)

print(f"正在將測試資料按測量類型儲存到: {output_base_dir}")

measurement_types = ['X', 'Y', 'Z', 'Voltage']

if 'testing_merged_df' not in locals() or testing_merged_df.empty:
    print("錯誤: testing_merged_df 不存在或為空，請確認之前的步驟已成功執行。")
else:
    for m_type in measurement_types:
        # 找出所有屬於當前測量類型的欄位
        # 並將這些欄位排序，以便輸出時順序一致
        data_cols_for_type = sorted([col for col in testing_merged_df.columns if col.endswith(f'_{m_type}')])

        # 創建一個新的DataFrame，只包含當前測量類型的所有數據 (不含 X_Value)
        df_m_type = testing_merged_df[data_cols_for_type].copy()

        # 定義輸出檔案路徑，使用新的命名格式
        output_file_name = f'Testing_Merge_Clean_Table_{m_type}.csv'
        output_file_path = os.path.join(output_base_dir, output_file_name)

        # 儲存為CSV檔案
        df_m_type.to_csv(output_file_path, index=False)
        print(f"已儲存 {m_type} 測試資料到: {output_file_path} (形狀: {df_m_type.shape})")

print("\n所有測試測量類型資料已分離並儲存完畢。")


--- 步驟 3: 分離並儲存乾淨的測試資料 (去除X_Value) ---
正在將測試資料按測量類型儲存到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data
已儲存 X 測試資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Testing_Merge_Clean_Table_X.csv (形狀: (30000, 24))
已儲存 Y 測試資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Testing_Merge_Clean_Table_Y.csv (形狀: (30000, 24))
已儲存 Z 測試資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Testing_Merge_Clean_Table_Z.csv (形狀: (30000, 24))
已儲存 Voltage 測試資料到: /content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data/Testing_Merge_Clean_Table_Voltage.csv (形狀: (30000, 24))

所有測試測量類型資料已分離並儲存完畢。


## 驗證儲存的測試資料

### Subtask:
載入並顯示每個分開儲存的 Testing CSV 檔案的前5行和後5行，以視覺化驗證資料的正確性和儲存結果。


**Reasoning**:
To visually verify the correctness and storage results of the testing data, I need to load each of the separately saved Testing CSV files and display their first and last five rows.



In [77]:
import pandas as pd
import os

print("--- 步驟 4: 驗證儲存的測試資料 ---")
print("正在載入並顯示每個儲存的測試 CSV 檔案的前5行和後5行：\n")

output_base_dir = '/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data'
measurement_types = ['X', 'Y', 'Z', 'Voltage']

for m_type in measurement_types:
    output_file_name = f'Testing_Merge_Clean_Table_{m_type}.csv'
    output_file_path = os.path.join(output_base_dir, output_file_name)

    if os.path.exists(output_file_path):
        print(f"--- {output_file_name} ---")
        df_loaded_testing = pd.read_csv(output_file_path)
        print("前5行：")
        display(df_loaded_testing.head())
        print("後5行：")
        display(df_loaded_testing.tail())
        print("\n")
    else:
        print(f"錯誤: 檔案 {output_file_path} 不存在。")

print("所有測試檔案檢查成功。")

--- 步驟 4: 驗證儲存的測試資料 ---
正在載入並顯示每個儲存的測試 CSV 檔案的前5行和後5行：

--- Testing_Merge_Clean_Table_X.csv ---
前5行：


,faulty_ (1)_X,faulty_ (10)_X,faulty_ (11)_X,faulty_ (12)_X,faulty_ (2)_X,faulty_ (3)_X,faulty_ (4)_X,faulty_ (5)_X,faulty_ (6)_X,faulty_ (7)_X,...,healthy_ (11)_X,healthy_ (12)_X,healthy_ (2)_X,healthy_ (3)_X,healthy_ (4)_X,healthy_ (5)_X,healthy_ (6)_X,healthy_ (7)_X,healthy_ (8)_X,healthy_ (9)_X
0,0.011817,0.031624,-0.010794,-0.020041,0.025030,-0.003860,-0.041964,0.012884,-0.008924,-0.059996,...,0.034202,0.010276,-0.045524,0.035339,0.051067,-0.015658,-0.010527,0.002689,0.009735,-0.024562
1,-0.017410,0.033610,0.005295,-0.013885,-0.007492,0.041209,-0.030039,-0.010473,0.003585,-0.096980,...,0.011538,-0.011364,-0.005067,0.011474,-0.004804,-0.021239,-0.000990,0.008907,0.006923,-0.041132
2,-0.027170,0.011436,0.010623,-0.008690,-0.025616,0.020464,0.011657,-0.022124,-0.003269,0.011165,...,-0.005219,-0.008745,0.031375,0.000919,0.023639,0.016198,0.017864,0.013894,0.028026,-0.028097
3,-0.022172,0.018251,-0.018272,-0.014563,-0.017450,-0.038271,0.029727,-0.021597,-0.005094,0.065965,...,-0.012283,0.017696,0.041163,-0.009511,0.017523,0.045082,0.017358,-0.000176,0.011430,0.017793
4,-0.016855,0.015280,0.001357,0.020366,0.005600,-0.033323,0.018417,-0.016835,-0.017271,0.096978,...,-0.003315,0.036480,-0.009538,-0.010027,0.019182,0.045744,-0.007930,0.015607,-0.012547,0.026768


後5行：


,faulty_ (1)_X,faulty_ (10)_X,faulty_ (11)_X,faulty_ (12)_X,faulty_ (2)_X,faulty_ (3)_X,faulty_ (4)_X,faulty_ (5)_X,faulty_ (6)_X,faulty_ (7)_X,...,healthy_ (11)_X,healthy_ (12)_X,healthy_ (2)_X,healthy_ (3)_X,healthy_ (4)_X,healthy_ (5)_X,healthy_ (6)_X,healthy_ (7)_X,healthy_ (8)_X,healthy_ (9)_X
29995,0.040295,0.028781,-0.026730,-0.025540,-0.002631,-0.033988,-0.000878,0.028485,0.016865,-0.029468,...,-0.009638,-0.004485,-0.010820,0.007388,0.001610,-0.002735,0.019343,0.007802,-0.005526,-0.006745
29996,0.046518,-0.010063,-0.023891,-0.011341,-0.011238,-0.008272,0.044406,0.012916,-0.006104,-0.006173,...,-0.018062,-0.030134,-0.022704,0.008237,-0.002920,-0.024907,-0.009170,-0.000387,-0.006739,-0.008190
29997,0.003400,-0.053879,-0.016675,-0.008236,0.006342,0.012178,0.054220,0.001007,-0.010188,0.028898,...,0.013799,-0.029145,-0.029886,0.019129,0.009600,-0.011207,-0.021298,-0.016068,0.004116,0.010403
29998,0.002564,-0.033671,-0.006432,-0.020792,0.026115,0.021446,-0.017664,0.004356,0.000118,0.035175,...,0.027885,-0.004344,0.019634,0.030685,-0.003320,-0.019597,-0.029368,-0.003774,-0.004280,0.006204
29999,0.022317,-0.015762,0.000519,-0.027790,0.023170,0.007456,-0.001688,-0.010599,0.016130,0.010401,...,0.019025,0.036499,0.051256,0.007146,0.018248,0.016392,-0.021427,-0.010051,0.003298,0.012435




--- Testing_Merge_Clean_Table_Y.csv ---
前5行：


...



--- Testing_Merge_Clean_Table_Voltage.csv ---
前5行：


,faulty_ (1)_Voltage,faulty_ (10)_Voltage,faulty_ (11)_Voltage,faulty_ (12)_Voltage,faulty_ (2)_Voltage,faulty_ (3)_Voltage,faulty_ (4)_Voltage,faulty_ (5)_Voltage,faulty_ (6)_Voltage,faulty_ (7)_Voltage,...,healthy_ (11)_Voltage,healthy_ (12)_Voltage,healthy_ (2)_Voltage,healthy_ (3)_Voltage,healthy_ (4)_Voltage,healthy_ (5)_Voltage,healthy_ (6)_Voltage,healthy_ (7)_Voltage,healthy_ (8)_Voltage,healthy_ (9)_Voltage
0,0.002991,0.001398,0.003947,0.001717,0.003310,0.043448,0.003947,0.002354,0.002672,-0.069323,...,0.001717,-0.063907,0.000124,0.001080,-0.079517,0.000124,0.003310,0.001080,0.059695,0.003628
1,0.003628,0.000443,0.003310,0.001717,0.003310,0.015415,0.004265,0.002035,0.002991,-0.092896,...,0.001398,-0.085888,0.001080,0.001080,-0.061677,0.000443,0.002672,0.001717,0.031662,0.003310
2,0.002991,0.000443,0.003947,0.001398,0.003310,0.001717,0.004265,0.002354,0.002035,-0.104683,...,0.000443,-0.101179,0.001398,0.000124,-0.043200,0.000443,0.002991,0.002035,0.005540,0.003310
3,0.002991,0.000443,0.004265,0.002035,0.003628,0.000761,0.004584,0.002672,0.002354,-0.102453,...,0.001398,-0.102453,0.001080,0.001398,-0.020264,0.000443,0.002991,0.002035,0.001080,0.002672
4,0.002991,0.000443,0.003628,0.002035,0.003628,0.001717,0.004584,0.003310,0.002672,-0.091303,...,0.001080,-0.091622,0.001717,0.000761,0.002991,-0.000832,0.002991,0.002035,0.000124,0.003310


後5行：


,faulty_ (1)_Voltage,faulty_ (10)_Voltage,faulty_ (11)_Voltage,faulty_ (12)_Voltage,faulty_ (2)_Voltage,faulty_ (3)_Voltage,faulty_ (4)_Voltage,faulty_ (5)_Voltage,faulty_ (6)_Voltage,faulty_ (7)_Voltage,...,healthy_ (11)_Voltage,healthy_ (12)_Voltage,healthy_ (2)_Voltage,healthy_ (3)_Voltage,healthy_ (4)_Voltage,healthy_ (5)_Voltage,healthy_ (6)_Voltage,healthy_ (7)_Voltage,healthy_ (8)_Voltage,healthy_ (9)_Voltage
29995,0.001080,-0.003699,0.001717,0.004265,0.003947,0.003628,0.126593,0.003947,0.002035,0.003310,...,0.000443,0.002035,0.002991,0.001080,0.001717,-0.054669,0.000761,0.002035,0.002035,0.125956
29996,0.002035,0.004265,0.001398,0.004265,0.004584,0.002991,0.117673,0.003947,0.002672,0.003310,...,0.000124,0.001717,0.002991,0.001398,0.001398,-0.081109,0.000124,0.003947,0.001717,0.123089
29997,0.001717,0.004265,0.001717,0.004584,0.003947,0.002991,0.103338,0.004584,0.002035,0.002354,...,0.000761,0.001080,0.002354,0.000443,0.000443,-0.095126,0.000443,0.003310,0.002035,0.112258
29998,0.001080,0.003947,0.002035,0.003947,0.003628,0.002991,0.085180,0.004902,0.002354,0.002991,...,0.000443,0.000443,0.002354,0.001080,0.000443,-0.100860,0.000443,0.002672,0.001717,0.095055
29999,0.000443,0.003947,0.001080,0.004265,0.002991,0.002672,0.067978,0.004265,0.002354,0.001717,...,0.001717,0.000443,0.003947,0.000124,0.000761,-0.095445,0.001398,0.003628,0.001717,0.079446




所有測試檔案檢查成功。


## 最終任務

### Subtask:
匯報所有 Testing 資料的處理和儲存已完成。


## Summary:

### Q&A
The task implicitly asked to confirm the correctness and integrity of the processed and saved testing data.

The verification process successfully confirmed that:
*   Four CSV files (`Testing_Merge_Clean_Table_X.csv`, `Testing_Merge_Clean_Table_Y.csv`, `Testing_Merge_Clean_Table_Z.csv`, `Testing_Merge_Clean_Table_Voltage.csv`) were correctly created and saved in the `/content/drive/MyDrive/ML_MY_Project_FCUT/data/Clean_Data` directory.
*   Each of these files could be loaded without errors.
*   The displayed head and tail of the loaded DataFrames showed the expected column structure with file prefixes (e.g., `faulty_ (1)_X`), indicating that the merging, renaming, and separation steps were executed as intended.

### Data Analysis Key Findings
*   **LVM File Processing:** Successfully processed 24 LVM files from the `/content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing` directory.
*   **Merged DataFrame Creation:** A single `testing_merged_df` was created by merging all LVM files based on `X_Value`. This DataFrame had a shape of (30000, 97), comprising 30,000 data points and 97 columns (1 for `X_Value` and 4 measurements for each of the 24 files).
*   **Missing Value Handling:** No missing (NaN) values were detected in `testing_merged_df` either before or after the imputation step, indicating a clean merge.
*   **Cleaned Data Storage:** The full `testing_merged_df` was saved as `/content/drive/MyDrive/ML_MY_Project_FCUT/data/Testing/testing_merged_cleaned.csv`.
*   **Separated Data Storage:** The `testing_merged_df` was successfully separated into four distinct DataFrames, one for each measurement type (X, Y, Z, Voltage), and saved as individual CSV files (e.g., `Testing_Merge_Clean_Table_X.csv`). Each of these separated DataFrames had a shape of (30000, 24), containing 30,000 data points across the 24 original files for that specific measurement.
*   **Data Verification:** All separated and saved CSV files were successfully loaded and their structure visually confirmed, indicating correct data formatting and storage.

### Insights or Next Steps
*   The data preparation for the testing dataset is robust, with proper handling of file merging, column renaming, and verification of data integrity. This clean and structured data is ready for subsequent machine learning model testing or analysis.
*   Consider documenting the exact data ranges and statistical properties of each measurement type (X, Y, Z, Voltage) in the merged and separated datasets, which could be beneficial for feature engineering or model interpretation.
